In [3]:
from pygbif import occurrences
import geopandas as gpd
import pandas as pd


# Read back in (GeoPackage)
ferp_boundary = gpd.read_file("../data/ferp/ferp_boundary.gpkg", layer="ferp")


# Your list of tree species scientific names
tree_species = [
    "Sequoia sempervirens", "Quercus agrifolia", "Pseudotsuga menziesii",
    "Notholithocarpus densiflorus", "Arbutus menziesii", "Toxicodendron diversilobum",
    "Corylus cornuta", "Quercus parvula"
]

# Bounding box from your FERP boundary (use degrees)
ferp_bounds = ferp_boundary.to_crs("EPSG:4326").total_bounds
lon_min, lat_min, lon_max, lat_max = ferp_bounds  # (west, south, east, north)


In [5]:
from time import sleep

def get_gbif_occurrences(species, bbox, max_records=300):
    """
    Query GBIF for a given species and bounding box.
    """
    try:
        res = occurrences.search(
            scientificName=species,
            hasCoordinate=True,
            limit=max_records,
            decimalLatitude=f"{bbox[1]},{bbox[3]}",
            decimalLongitude=f"{bbox[0]},{bbox[2]}"
        )
        return pd.json_normalize(res['results'])
    except Exception as e:
        print(f"Failed for {species}: {e}")
        return pd.DataFrame()

# Pull occurrences for each tree species
gbif_records = []

for name in tree_species:
    print(f"Querying GBIF for {name}...")
    df = get_gbif_occurrences(name, ferp_bounds, max_records=500)
    df["species"] = name
    gbif_records.append(df)
    sleep(1)  # be kind to the API

gbif_df = pd.concat(gbif_records, ignore_index=True)
print("Total GBIF records pulled:", len(gbif_df))


Querying GBIF for Sequoia sempervirens...
Querying GBIF for Quercus agrifolia...
Querying GBIF for Pseudotsuga menziesii...
Querying GBIF for Notholithocarpus densiflorus...
Querying GBIF for Arbutus menziesii...
Querying GBIF for Toxicodendron diversilobum...
Querying GBIF for Corylus cornuta...
Querying GBIF for Quercus parvula...
Total GBIF records pulled: 0


In [ ]:
gbif_gdf = gpd.GeoDataFrame(
    gbif_df,
    geometry=gpd.points_from_xy(gbif_df["decimalLongitude"], gbif_df["decimalLatitude"]),
    crs="EPSG:4326"
).to_crs(grid.crs)  # Match EnMAP projection
